In [17]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [18]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [19]:
# Percentage of documents in 'spam_data' that are spam
print('{} %'.format(round((len(spam_data[spam_data['target']==1])/len(spam_data))*100 ,4))) 

13.4063 %


In [20]:
# longest token in the vocabulary

from sklearn.feature_extraction.text import CountVectorizer

vect=CountVectorizer().fit(X_train)
df=pd.DataFrame([(i, len(i)) for i in vect.get_feature_names()])
# >>>type(df.iloc[0,1])
# numpy.int64

df[df[1]==df[1].max()] 

,0,1
1813,com1win150ppmx3age16subscription,32


In [21]:
# Fist, let's fit and transform the training data X_train using a Count Vectorizer 
# with default parameters. 
# Then we will use Naive Bayes classifier model with smoothing alpha=0.1 and will get its 
# AUC score

from sklearn import naive_bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score

vect=CountVectorizer().fit(X_train)
X_train_vectorized=vect.transform(X_train)

clfrNB=naive_bayes.MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)

y_pred=clfrNB.predict(vect.transform(X_test))

print('AUC score: {}'.format(round(roc_auc_score(y_test, y_pred), 4))) 

AUC score: 0.9721


In [22]:
# Now, let's return the 20 words (which are our features) with the smallest tf-idf and 
# the 20 words with the largest tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer

    
vect=TfidfVectorizer().fit(X_train)
feature_names=np.array(vect.get_feature_names())
sorted_index_values=vect.transform(X_train).max(0).toarray().reshape(-1,).argsort()

tf_idfs=vect.transform(X_train).max(0).toarray().reshape(-1,)

smallest_idfs=pd.DataFrame(data={'values':[tf_idfs[i] for i in sorted_index_values[:20]], 
                           'name':[feature_names[i] for i in sorted_index_values[:20]]}).sort_values(by=['values','name']).set_index('name')

largest_idfs=pd.DataFrame(data={'values':[tf_idfs[i] for i in sorted_index_values[-21:-1]], 
                           'name':[feature_names[i] for i in sorted_index_values[-21:-1]]}).sort_values(by=['values','name'], ascending=False).set_index('name')


smallest_idfs=pd.Series(data=smallest_idfs.iloc[:,0], index=smallest_idfs.index)
smallest_idfs.name=None
smallest_idfs.index.name=None

largest_idfs=pd.Series(data=largest_idfs.iloc[:,0], index=largest_idfs.index)
largest_idfs.name=None
largest_idfs.index.name=None

(smallest_idfs, largest_idfs)

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, yup         1.000000
 where       1.000000
 too         1.000000
 thanx       1.000000
 thank       1.000000
 okie        1.000000
 ok          1.000000
 nite        1.000000
 lei         1.000000
 home        1.000000
 havent      1.000000
 er          1.000000
 done        1.000000
 beerage     1.000000
 anytime     1.000000
 anything    1.000000
 645         1.000000
 tick        0.980166
 blank       0.932702
 same        0.932467
 dtype: float64)

In [23]:
# average length of documents (length based on number of characters) for not spam and 
# spam documents

len_not_spam=pd.DataFrame(data=[len(spam_data['text'].iloc[i]) for i in range(len(spam_data)) if spam_data['target'].iloc[i]==0]).mean()

len_spam=pd.DataFrame(data=[len(spam_data['text'].iloc[i]) for i in range(len(spam_data)) if spam_data['target'].iloc[i]==1]).mean()

(float(len_not_spam), float(len_spam))

(71.02362694300518, 138.8661311914324)